In [ ]:
import numpy as np
import trimesh
from ac3D.snake3D import evolution_snake3d, Snake3DParams
from ac3D.meshing import trisphere
from ac3D.visualization import visualize_evolution, visualize_volume_plotly
import scipy.ndimage as ndi
import plotly.graph_objects as go


In [ ]:
size = 100
volume = np.zeros((size, size, size))  # background is 0

# Ellipsoid axes (semi-axis lengths)
a, b, c = size // 3, size // 4, size // 5  # along x, y, z

# Generate coordinate grid
x, y, z = np.ogrid[-size/2:size/2, -size/2:size/2, -size/2:size/2]

# Ellipsoid equation
mask = (x / a)**2 + (y / b)**2 + (z / c)**2 <= 1

# Fill ellipsoid
volume[mask] = 1

# Reduce noise and smoothing to create sharper boundaries
# volume = ndi.gaussian_filter(volume + 0.1 * np.random.randn(*volume.shape), sigma=1)

In [ ]:
# # # Example usage
# fig = visualize_volume_plotly(volume, threshold=0.1)
# fig.show()

In [ ]:
# Create initial mesh (slightly larger than the object)
vertices, faces = trisphere(iterations=5, radius=10, x_0=size//2, y_0=size//2, z_0=size//2)
mesh_init = trimesh.Trimesh(vertices=vertices, faces=faces)

In [ ]:
#mesh_init.show()

In [ ]:
# In a notebook cell:
mesh_snapshots = []

# Save initial mesh
mesh_snapshots.append(mesh_init.copy())

# # Run snake for 250 iterations
# mesh_1 = evolution_snake3d(
#     volume, 
#     mesh_init,
#     alpha=3000.0,
#     beta=1000.0,
#     gamma=3e-5,
#     kb=150.0,
#     vfc_size=20,        # Larger kernel 
#     vfc_sigma=7.0,      # Wider spread
#     max_iter=400
# )
# mesh_snapshots.append(mesh_1)

# mesh_2 = evolution_snake3d(
#     volume, 
#     mesh_init,
#     alpha=3000.0,
#     beta=1000.0,
#     gamma=3e-5,
#     kb=150.0,
#     vfc_size=20,        # Larger kernel 
#     vfc_sigma=7.0,      # Wider spread
#     max_iter=800
# )
# mesh_snapshots.append(mesh_2)

mesh_3 = evolution_snake3d(
    volume, 
    mesh_init,
    alpha=3500.0,
    beta=500.0,
    sigma=1.0,
    gamma=3e-4,
    kb=150.0,
    vfc_size=15,        # Larger kernel 
    vfc_sigma=5.0,      # Wider spread
    max_iter=3000,
    remesh_interval=250
)
mesh_snapshots.append(mesh_3)

In [ ]:
# Create and display the visualization
fig = visualize_evolution(volume, mesh_snapshots, threshold=0.5)
fig.show()

In [ ]:
# In a notebook cell:
mesh_snapshots = []

# Save initial mesh
mesh_snapshots.append(mesh_init.copy())

# Run snake for 250 iterations
mesh_1 = evolution_snake3d(
    volume, 
    mesh_init,
    alpha=1500.0,
    beta=170.0,
    gamma=3e-5,
    kb=300.0,
    vfc_size=5,        # Larger kernel 
    vfc_sigma=3.0,      # Wider spread
    max_iter=400
)
mesh_snapshots.append(mesh_1)

mesh_2 = evolution_snake3d(
    volume, 
    mesh_init,
    alpha=1500.0,
    beta=170.0,
    gamma=3e-5,
    kb=300.0,
    vfc_size=5,        # Larger kernel 
    vfc_sigma=3.0,      # Wider spread
    max_iter=800
)
mesh_snapshots.append(mesh_2)

mesh_3 = evolution_snake3d(
    volume, 
    mesh_init,
    alpha=1500.0,
    beta=170.0,
    gamma=3e-5,
    kb=300.0,
    vfc_size=5,        # Larger kernel 
    vfc_sigma=3.0,      # Wider spread
    max_iter=1600
)
mesh_snapshots.append(mesh_3)

In [ ]:
# Create and display the visualization
fig2 = visualize_evolution(volume, mesh_snapshots, threshold=0.5)
fig2.show()